In [1]:
"""import packages"""
import os, glob
import pandas as pd
import numpy as np
import math, random
from datetime import datetime
import pickle
from matplotlib import pyplot as plt
import argparse
import path, sys, re, time
from collections import Counter

In [2]:
from module_.featureExtraction import feature_extraction
from module_.changePointDetection import change_point_detection

In [6]:
# load ppsed data
events=np.load("./preprocessed/hh/hh101/pp.npy")
# print(events.shape)

# unique sensor list
sensor_list=sorted(set(events[:,0]))
# print(sensor_list)

# transition idx: start of next activity
trs=[i for i in range(events.shape[0]-1) if events[i,-1]!=events[max(i-1, 0),-1]]
print(len(trs), trs[:2])
print(events[240:250])

# features=feature_extraction(events, 'hh101', sensor_list)
# scores, _, _, _ = change_point_detection(features, 'rulsif', save=True)


4456 [240, 248]
[['MA015' 'ON' '1342750158.9523' 'Toilet']
 ['MA015' 'ON' '1342750161.252048' 'Toilet']
 ['MA015' 'ON' '1342750171.387414' 'Toilet']
 ['MA015' 'ON' '1342750176.565344' 'Toilet']
 ['MA015' 'ON' '1342750182.242239' 'Toilet']
 ['MA015' 'ON' '1342750185.642422' 'Toilet']
 ['MA015' 'ON' '1342750192.714222' 'Toilet']
 ['MA015' 'ON' '1342750194.43453' 'Toilet']
 ['MA015' 'ON' '1342750201.751574' '']
 ['M011' 'ON' '1342750207.430253' '']]


In [3]:
"""sliding window"""
ws=30
windows=[]
for i in range(events.shape[0]):
    window=events[i-ws+1:i+1,:]
    if i-ws+1<0:
        repeat=np.array([events[0,:] for j in range(ws-i-1)])
        window=np.concatenate((repeat,events[:i+1]), axis=0)
    windows.append(window)
windows=np.array(windows)
windows.shape

(219513, 30, 4)

In [6]:
"""feature extraction and ATD
    * feature space is updated when a new event occurs
        - min and max for normalization is updated, so feature value is also updated?
    * dissimilarity measure between x(t) and x(t+n)
    * x(t)=[f(t-(n-1),..., f(t)] whose length is n
    * x(t+n)=[f(t+n-(n-1)),..., f(t+n)] whose length is n
    in case of n=2, x(t)=[f(t-1), f(t)] and x(t+2)=[f(t+1), f(t+2)]
"""
n=2
max_wd=0.
scores=[]
start=time.time()
features=[]
for i in range(windows.shape[0]):
    tn1=max(i-1, 0); tp1=min(windows.shape[0]-1, i+1); tp2=min(windows.shape[0]-1, i+2)
    samples=np.array([windows[tn1], windows[i], windows[tp1], windows[tp2]]).reshape((4, ws, -1))
    samples_n, max_wd=wtfr(samples, sensor_set, max_wd)
    features.append(samples_n)
    x_t=np.array(samples_n[:2]).reshape((2, -1))
    x_tn=np.array(samples_n[2:]).reshape((2, -1))

    dre=dr(x=x_t, y=x_tn, alpha=0.5)
    scores.append(dre.PEDiv)

    if i!=0 and i%500==0:
        print("{}/{} elapsed time: {}".format(i, windows.shape[0], time.time()-start))
        np.save("scores/casas_rulsif.npy", scores)

500/219513 elapsed time: 582.9997804164886
1000/219513 elapsed time: 1159.6860611438751
1500/219513 elapsed time: 1737.7762799263
2000/219513 elapsed time: 2317.0540986061096
2500/219513 elapsed time: 2894.4299731254578
3000/219513 elapsed time: 3470.786099433899
3500/219513 elapsed time: 4049.1391830444336
4000/219513 elapsed time: 4628.603989601135
4500/219513 elapsed time: 5207.53546333313
5000/219513 elapsed time: 5787.6814057827
5500/219513 elapsed time: 6367.925613641739
6000/219513 elapsed time: 6946.541858673096
6500/219513 elapsed time: 7522.8583290576935
7000/219513 elapsed time: 8102.630624055862
7500/219513 elapsed time: 8682.065217971802
8000/219513 elapsed time: 9258.9493496418
8500/219513 elapsed time: 9835.588384628296
9000/219513 elapsed time: 10416.636788129807
9500/219513 elapsed time: 10993.707657575607
10000/219513 elapsed time: 11570.96715927124
10500/219513 elapsed time: 12148.590377569199
11000/219513 elapsed time: 12728.587332248688
11500/219513 elapsed time: 1

KeyboardInterrupt: 

In [4]:
load_scores=np.load("scores/casas_rulsif.npy"); len(load_scores)

40001

In [6]:
# trs[417] = 38961
events_=events[:38962]
scores_=load_scores[:38962]
trs_=trs[:418]

final_score=[]
for i in range(len(scores_)):
    prev1=max(0, i-1); prev2=max(0, i-2)
    final_score.append(scores_[prev1]+scores_[prev2])

threshold=0.9
candidates=[]
"""minimum distance between two transitions=1"""
# from scipy.signal import find_peaks
# peaks, _=find_peaks(final_score, distance=1)

# for item in peaks:
#     if final_score[item]>threshold:
#         candidates.append(i)
# len(candidates)

idx=0
for i in range(len(final_score)):
    if final_score[i]>threshold:
        if abs(i-idx)==1:
            continue
        else:
            candidates.append(i)
            idx=i

In [8]:
trs_t=np.array([float(events[idx][-2]) for idx in trs_])
trs_t_=np.array([float(events[idx+1][-2]) for idx in trs_])
tp, tn, fp, fn=[],[],[],[]
for i in range(len(events_)):
    if i in candidates: # positive
        if i in trs_:
            tp.append(i)
            continue
        timestamp_=float(events_[i][-2])
        if sum(abs(trs_t-timestamp_)<15)!=0:
            tp.append(i)
            continue
        if sum(abs(trs_t_-timestamp_)<15)!=0:
            tp.append(i)
            continue
        fp.append(i)
    else:
        if i in trs_:
            fn.append(i)
            continue
        tn.append(i)
tp_, tn_, fp_, fn_=len(tp), len(tn), len(fp), len(fn)

print(tp_/(tp_+fn_))
print(tn_/(tn_+fp_))

print(tp[0])

0.19298245614035087
0.9836926838149237
1779
